# Satisfação de clientes

Para qualquer empresa que atua diretamente com seus clientes, conhecer o grau de satisfação destes é fundamental. Clientes insatisfeitos com os serviços prestados não permanecem por muito tempo e acabam procurando empresas concorrentes para atender suas necessidades. Em muitos casos, estes clientes não expressam suas insatisfações, o que dificulta a obtenção de feedback por parte da empresa.

A partir de um conjunto de dados devidamente etiquetado, podemos combinar as variáveis disponíveis com a finalidade de identificar padrões que permitam rotular novos clientes. Em outras palavras, **podemos utilizar dados dos próprios clientes para identificar seu grau de satisfação**.

Neste trabalho, vamos utilizar os dados disponibilizados pelo banco Santander para elaborar um modelo de Machine Learning capaz de identificar os clientes satisfeitos e insatisfeitos.

# Análise dos dados disponíveis

In [1]:
#Libs
import pandas as pd

#Dataset
df = pd.read_csv('../input/train.csv')

#Formato
print('Formato dos dados disponíveis: %s' % (df.shape,))

#Proporção satisfeitos/insatisfeitos
print('Total de clientes satisfeitos: %.0f' % len(df[df.TARGET == 0]))
print('Total de clientes insatisfeitos: %.0f' % len(df[df.TARGET == 1]))

Formato dos dados disponíveis: (76020, 371)
Total de clientes satisfeitos: 73012
Total de clientes insatisfeitos: 3008


In [2]:
#Exibindo algumas amostras
df.sample(5)

,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
54298,108323,2,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,104900.64,0
44515,89130,2,27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,84937.80,0
31552,63074,2,24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,71843.79,0
14800,29774,2,29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,54816.69,1
7887,15889,2,27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,83325.57,0


1. Temos 76020 registros de clientes e 371 variáveis contendo diferentes informações sobre cada registro.
2. Dos 76020 clientes, apenas 3008 foram previamente classificados como insatisfeitos, sugerindo que é muito mais provável identificar clientes satisfeitos

## 1. Seleção de variáveis

Para conhecer cada variável e selecionar as mais significativas entre as 371, utilizei a biblioteca *pandas-profiling*. Os seguintes critérios de seleção foram estabelecidos:

* Desconsiderar variáveis constantes;
* Desconsiderar variáveis altamente correlatas (p >= 0.90). Nestas situações, apenas uma variável foi escolhida.
* Desconsiderar variáveis com mais de 90% dos valores preenchidos por zeros.

Embora o Santander não tenha fornecido nenhuma explicação sobre cada variável, é possível conduzir análises estatísticas sobre estes dados para extrair informações. O processo de análise que desenvolvi está disponível <a href='00-pandasProfilingEDA.html'>nesta página</a>. Listo abaixo as variáveis selecionadas para compor o modelo preditivo:

* Variáveis binárias
    + ind_var30
    + ind_var39_0
    + ind_var43_emit_ult1
    + ind_var43_recib_ult1
    + ind_var5
    + ind_var5_0
* Variáveis categóricas
    + num_meses_var12_ult3
    + num_meses_var13_corto_ult3
    + num_meses_var39_vig_ult3
    + num_meses_var5_ult3
    + num_meses_var8_ult3
* Variáveis numéricas
    + imp_aport_var13_hace3
    + imp_trans_var37_ult1
    + saldo_var30

## 2. Balanceamento de amostras

O algoritmo de Machine Learning dificilmente será capaz de compreender as características dos clientes se a maior parte dos registros representar clientes satisfeitos. Existem duas abordagens para solucionar este problema:

* Atribuímos um peso maior à classificação correta de clientes insatisfeitos
* Selecionamos uma sub-amostra balanceada (50% satisfeitos, 50% insatisfeitos)

Cada algoritmo possui uma abordagem mais adequada. Para este problema, optei por implementar árvores de decisão com Random Forest e a segunda opção foi a alternativa mais adequada.

# Criação do modelo preditivo

Existem diversos algoritmos capazes de solucionar este problema. Neste projeto, optei por utilizar a implementação RandomForestClassifier da biblioteca *sklearn*. Essencialmente, um algoritmo RandomForest efetua a implementação de múltiplas árvores de decisão, extraindo as características mais relevantes de cada árvore para implementar um modelo robusto e abrangente.

Neste documento, descrevo resumidamente os procedimentos executados e resultados obtidos. A implementação completa pode ser visualizada <a href='01-buildModel.html'>nesta página</a>

## 1. Pipeline de pré-processamento

A partir das variáveis selecionadas, foram realizadas as seguintes operações:

* Variáveis categóricas: transformação dos dados com OneHotEncoding, de forma a redimensionar este conjunto de variáveis em formato binário.
* Variáveis numéricas: transformação dos dados com MinMaxScaler, de forma a colocar estas variáveis numa escala de valores entre 0 e 1.
* Variáveis booleanas: nenhuma transformação, utilizando-as no formato original.

As demais variáveis do dataset foram descartadas.

## 2. Métricas de avaliação

Selecionei as métricas listadas abaixo para avaliar a performance do modelo desenvolvido

* **Accuracy:** Para determinar a capacidade do modelo em prever corretamente clientes satisfeitos e insatisfeitos.
* **Recall:** Para determinar a capacidade do modelo em identificar corretamente os clientes insatisfeitos
* **ROC-AUC:** Para calcular a área abaixo da curva ROC, obtendo assim uma compreensão generalizada da capacidade do modelo
    + Esta métrica realiza um comparativo entre verdadeiros positivos e falsos positivos, variando entre 0 e 1. Um score igual a 0 indica que o modelo errou todas as previsões, enquanto que um score igual a 1 indica que o modelo acertou todas as previsões. Um score de 0.5 sugere que as previsões realizadas pelo modelo foram realizadas de maneira aleatória.

## 3. Resultados obtidos: *baseline*

O dataset utilizado para treino e validação consiste de uma sub-amostra selecionada aleatoriamente do dataset original balanceada 50/50 entre clientes satisfeitos e insatisfeitos.

Para testar o modelo num cenário realista, selecionei 30.000 registros aleatoriamente do dataset original.

* **Validação**
    + Accuracy: 68%
    + Recall: 70%
    + ROC-AUC: 68%
* **Teste**
    + Accuracy: 66% (-2%)
    + Recall: 79% (+9%)
    + ROC-AUC: 72% (+4%)
    
Apesar da redução em Accuracy, o modelo apresentou uma boa capacidade em identificar corretamente mais clientes insatisfeitos.

## 4. Otimização de hiperparâmetros

Em busca de melhorar a performance do modelo *baseline*, implementei dois novos modelos RandomForest com apoio de um algoritmo de suporte GridSearchCV, cuja finalidade é implementar diferentes combinações de hiperparâmetros em busca de uma solução ótima.

Foram ajustados os seguintes parâmetros:

* n_estimators: para identificar a quantidade adequada de árvores a implementar
* max_depth: para identificar a quantidade adequada de variáveis a utilizar na implementação de cada árvore
* min_samples_split: para identificar o volume adequado de amostras necessárias para estabelecer pontos de decisão
* min_samples_leaf: para identificar o volume adequado de amostras para efetuar uma decisão

## 5. Novos resultados

Selecionei as métricas Accuracy e Recall como métodos de otimização do modelo. Segue abaixo os resultados obtidos, respectivamente, bem como o comparativo com os resultados de teste do modelo *baseline*:

* Modelo cv_acc:
    + Accuracy: 68% (+2%)
    + Recall: 76% (-3%)
    + ROC-AUC: 72% (+0%)
* Modelo cv_rec:
    + Accuracy: 66% (0%)
    + Recall: 82% (+3%)
    + ROC-AUC: 73% (+1%)
    
O modelo 'cv_acc' apresentou uma melhoria em classificar clientes satisfeitos ao custo de uma pequena redução na capacidade de classificar clientes insatisfeitos. Já o modelo 'cv_rec' manteve a mesma Accuracy observada anteriormente e obteve uma melhora na capacidade de identificar clientes insatisfeitos.

## 6. Submetendo resultados ao Kaggle

Por fim, utilizei os dados de teste da competição para analisar a performance dos três modelos quando confrontados com dados totalmente inéditos.

![kaggle_submission](kaggle_submission.png)

O modelo 'cv_acc' apresentou a melhor performance entre os três, embora a performance destes tenha sido bastante similar.